In [1]:
import pandas as pd

image_list = pd.read_csv("data/raw_image_list.csv")
image_list

,User,Date,Isilon path,Single CZI,File Name,Experiment Type,QC?,Diff?,Cell Line,CL/CR#,...,Fluorescent Protein,Other Label,Drug/Treatment,Microscope,Objective,Dimension,MP?,Time-lapse,Exp. Details,Other
0,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001.czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,41.5hr long movie of lamin,cells not fed during imaging but look healthy
1,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(1).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 1",cells not fed during imaging but look healthy
2,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(2).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 2",cells not fed during imaging but look healthy
3,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(3).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 3",cells not fed during imaging but look healthy
4,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(4).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 4",cells not fed during imaging but look healthy
5,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(5).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 5",cells not fed during imaging but look healthy
6,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(6).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 6",cells not fed during imaging but look healthy
7,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(7).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 7",cells not fed during imaging but look healthy
8,CCH,2018.03.29,/aics/assay-dev/MicroscopyData/Caroline/2018/2...,Yes,20180329_C01_001.czi,AdditionalWorkup,NaN,cardiomyocyte,37.0,NaN,...,NaN,NaN,NaN,ZSD0,40x/1.2W,3D,Yes,Yes,"Mega-file, includes dividing cardiomyocyte",NaN
9,CCH,2018.07.24,/aics/assay-dev/MicroscopyData/Caroline/2018/2...,Yes,20180724_C01_001.czi,AdditionalWorkup,NaN,ipsc,0.0,NaN,...,NaN,NaN,NaN,zsd0,100x/1.25W,3D,no,yes,tl timelapses for modeling; see txt,NaN


In [2]:
from pathlib import Path
import shutil

remote = (Path("/allen") / image_list.loc[1]["Isilon path"][1:].replace("assay-dev", "assay_dev") / image_list.loc[1]["File Name"]).resolve(strict=True)
print(f"Remote filepath: {remote}, Size in GB: {remote.stat().st_size / 1000 / 1000 / 1000}")

# local = Path(f"/tmp/{czi.name}")
# print(f"Beginning copy to: {local}")
# shutil.copy(str(czi), str(local))
# local.resolve(strict=True)
# print(f"Copy complete, located at: {local}")

Remote filepath: /System/Volumes/Data/allen/aics/assay_dev/MicroscopyData/Caroline/2017/20170703/20170703_C01_001(1).czi, Size in GB: 83.96012224


In [3]:
from aicspylibczi import CziFile
import numpy as np
from PIL import Image
import imageio

# Init czi
img = CziFile(remote)

# Start the movie writer
movie_target = Path("~/Desktop/test.mp4")
writer = imageio.get_writer(str(movie_target), fps=1)

# Iterate over time dim
for i in range(img.dims()["T"][1])[:5]:
    print(f"Reading timepoint: {i}...")
    loaded_t, shape = img.read_image(T=i, B=0, S=0)
    print("Generating projection...")
    
    # Some weird normalization by percentiale and then clip we found through testing and by eye
    norm_by = np.percentile(loaded_t[0, 0, 0,], [50, 99.8])

    # Scale the numpy array values and cast them back to integers in the 0 to 255 range
    clipped = np.clip((loaded_t - norm_by[0]) / (norm_by[1] - norm_by[0]), 0, 1) * 255

    # Just get the max project of the z for the first channel (nuclear membrane dye)
    proj = clipped[0, 0, 0].max(axis=0).astype(np.uint8)
    
    # Append projection
    writer.append_data(proj)

# Close the writer
writer.close()

/System/Volumes/Data/allen/aics/assay_dev/MicroscopyData/Caroline/2017/20170703/20170703_C01_001(1).czi
Read timepoint: 0, received shape: (1, 1, 2, 40, 1024, 1024)
Generating projection...
Read timepoint: 1, received shape: (1, 1, 2, 40, 1024, 1024)
Generating projection...
Read timepoint: 2, received shape: (1, 1, 2, 40, 1024, 1024)
Generating projection...
Read timepoint: 3, received shape: (1, 1, 2, 40, 1024, 1024)
Generating projection...
Read timepoint: 4, received shape: (1, 1, 2, 40, 1024, 1024)
Generating projection...
